# 요구사항 1. titanic_dataset.py 분석

In [1]:
import os
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader, random_split

In [2]:
class TitanicDataset(Dataset):
    def __init__(self, X, y):
        # 입력 데이터와 target 데이터를 나타내는 Numpy 배열을 FloatTensor와 LongTensor로 X와 y에 저장
        self.X = torch.FloatTensor(X)
        self.y = torch.LongTensor(y)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        feature = self.X[idx]
        target = self.y[idx]
        return {'input': feature, 'target': target}

    def __str__(self):
        str = "Data Size: {0}, Input Shape: {1}, Target Shape: {2}".format(
            len(self.X), self.X.shape, self.y.shape
        )
        return str

In [3]:
class TitanicTestDataset(Dataset):
    def __init__(self, X):
        # 입력 데이터 X를 FloatTensor로 X에 저장
        self.X = torch.FloatTensor(X)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        feature = self.X[idx]
        return {'input': feature}

    def __str__(self):
        str = "Data Size: {0}, Input Shape: {1}".format(
            len(self.X), self.X.shape
        )
        return str

### 정리 (1)
- TitanicDataset 과 TitanicTestDataset은 PyTorch의 Dataset 클래스를 상속하고 있다.
- 또한, 2개의 클래스는 훈련 데이터셋과 테스트 데이터셋을 나타내기 위한 기본적인 메서드들을 구현하고 있다.

In [4]:
def get_preprocessed_dataset():
    # 현재 스크립트 파일이 위치한 경로를 바탕으로 데이터 파일의 경로 설정
    CURRENT_FILE_PATH = os.path.dirname("hw2.ipynb")
    
    train_data_path = os.path.join(CURRENT_FILE_PATH, "train.csv")
    test_data_path = os.path.join(CURRENT_FILE_PATH, "test.csv")

    train_df = pd.read_csv(train_data_path)
    test_df = pd.read_csv(test_data_path)
    
    # 훈련 데이터와 테스트 데이터를 정렬하지 않고 병합하여 하나의 데이터 프레임으로 생성
    all_df = pd.concat([train_df, test_df], sort=False)

    # get_preprocessed_dataset_1 ~ 6을 호출하며 각각의 데이터 전처리 과정을 거쳐 수정된 데이터 프레임을 반환
    all_df = get_preprocessed_dataset_1(all_df)

    all_df = get_preprocessed_dataset_2(all_df)

    all_df = get_preprocessed_dataset_3(all_df)

    all_df = get_preprocessed_dataset_4(all_df)

    all_df = get_preprocessed_dataset_5(all_df)

    all_df = get_preprocessed_dataset_6(all_df)

    # === 데이터셋 생성 ===
    # train_X -> "Survived" 열을 제외한 나머지 훈련 데이터의 특성을 포함
    train_X = all_df[~all_df["Survived"].isnull()].drop("Survived", axis=1).reset_index(drop=True)
    # train_y -> "Survived" 열을 추출해 훈련 데이터의 타겟으로 사용
    train_y = all_df["Survived"]

    # test_X -> "Survived" 열이 결측치인 행들에서 "Survived" 열을 제외한 테스트 데이터셋의 입력 데이터 생성
    test_X = all_df[all_df["Survived"].isnull()].drop("Survived", axis=1).reset_index(drop=True)

    # 훈련 데이터셋 객체 생성
    dataset = TitanicDataset(train_X.values, train_y.values)
    #print(dataset)
    # 데이터셋을 훈련 데이터 : 검증 데이터 -> 8 : 2 비율로 랜덤하게 나누어서 저장 
    train_dataset, validation_dataset = random_split(dataset, [0.8, 0.2])
    # 테스트 데이터셋 객체 생성
    test_dataset = TitanicTestDataset(test_X.values)
    #print(test_dataset)

    return train_dataset, validation_dataset, test_dataset

### 정리 (2)
위 코드는 아래와 같은 기능을 한다.
- 훈련 데이터와 테스트 데이터셋을 전처리한다.
- 전처리한 데이터를 바탕으로 데이터셋 객체를 생성한다.
- 훈련 데이터는 훈련 데이터와 검증 데이터로 분배한다.
- 생성한 데이터셋 객체를 반환한다.

In [5]:
def get_preprocessed_dataset_1(all_df):
    # Pclass별 Fare 평균값을 사용하여 Fare 결측치 메우기
    # Pclass(티켓 클래스)와 Fare(운임 요금) 열을 all_df에서 선택 후 Pclass로 그룹화하여 Fare의 평균을 구한다.
    Fare_mean = all_df[["Pclass", "Fare"]].groupby("Pclass").mean().reset_index()
    # Pclass와 Fare 2개의 열을 생성한다.
    Fare_mean.columns = ["Pclass", "Fare_mean"]
    # all_df의 Pclass 열을 기준으로 all_df와 Fare_mean을 병합한다. 이 과정을 통해 원본 데이터에 Fare 열이 추가된다.
    all_df = pd.merge(all_df, Fare_mean, on="Pclass", how="left")
    # Fare 열의 값이 결측치인 경우 Fare 값을 동일한 Pclass의 Fare_mean(평균 운임 요금)으로 대체한다.
    all_df.loc[(all_df["Fare"].isnull()), "Fare"] = all_df["Fare_mean"]
    
    return all_df

In [6]:
def get_preprocessed_dataset_2(all_df):
    # name을 세 개의 컬럼으로 분리하여 다시 all_df에 합침
    # 기존 name을 ,.(쉼표 or 마침표)로 구분하여 (최대 2번) 새로운 데이터 프레임(expand=True가 여기에 해당)으로 반환한다.
    name_df = all_df["Name"].str.split("[,.]", n=2, expand=True)
    # 새로운 데이터 프레임의 컬럼 이름을 생성한다. (가족 이름 즉, 성, 경칭, 이름)
    name_df.columns = ["family_name", "honorific", "name"]
    # 문자열의 양쪽 공백을 제거한다.
    name_df["family_name"] = name_df["family_name"].str.strip()
    name_df["honorific"] = name_df["honorific"].str.strip()
    name_df["name"] = name_df["name"].str.strip()
    # 새롭게 생성한 3개의 열을 all_df와 병합한다.
    all_df = pd.concat([all_df, name_df], axis=1)

    return all_df

In [7]:
def get_preprocessed_dataset_3(all_df):
    # honorific별 Age 평균값을 사용하여 Age 결측치 메우기
    # honorific과 Age 컬럼을 선택하고 honorific을 기준으로 Age의 중앙값을 계산한다.. 그리고 이것을 반올림한다.
    honorific_age_mean = all_df[["honorific", "Age"]].groupby("honorific").median().round().reset_index()
    # 중앙값을 가지고 있는 데이터 프레임을 생성한다.
    honorific_age_mean.columns = ["honorific", "honorific_age_mean", ]
    # 원본 데이터와 중앙값 데이터 프레임을 honorific을 기준으로 병합한다.
    all_df = pd.merge(all_df, honorific_age_mean, on="honorific", how="left")
    # Age가 결측치인 값을 honorific별로 계산된 Age의 중앙값으로 대체한다.
    all_df.loc[(all_df["Age"].isnull()), "Age"] = all_df["honorific_age_mean"]
    # 더이상 사용하지 않는 중앙값 컬럼을 제거한다.
    all_df = all_df.drop(["honorific_age_mean"], axis=1)

    return all_df

In [8]:
def get_preprocessed_dataset_4(all_df):
    # 가족수(family_num) 컬럼 새롭게 추가
    # 부모/자녀 수와 형제/자매 수의 합으로 가족 수 컬럼을 계산
    all_df["family_num"] = all_df["Parch"] + all_df["SibSp"]

    # 혼자탑승(alone) 컬럼 새롭게 추가
    # 가족 수가 0인 경우 1로 설정, 아닌 경우 결측치로 남겨둔다.
    all_df.loc[all_df["family_num"] == 0, "alone"] = 1
    # 결측치는 0으로 채운다.
    all_df["alone"].fillna(0, inplace=True)

    # 학습에 불필요한 컬럼 제거
    all_df = all_df.drop(["PassengerId", "Name", "family_name", "name", "Ticket", "Cabin"], axis=1)

    return all_df

In [9]:
def get_preprocessed_dataset_5(all_df):
    # honorific 값 개수 줄이기
    # "Mr", "Miss", "Mrs", "Master"를 제외한 honorific 값을 "other"로 변경한다.
    all_df.loc[
    ~(
            (all_df["honorific"] == "Mr") |
            (all_df["honorific"] == "Miss") |
            (all_df["honorific"] == "Mrs") |
            (all_df["honorific"] == "Master")
    ),
    "honorific"
    ] = "other"
    
    # "Embarked"(승선항구) 컬럼의 결측치를 "missing"으로 대체한다.
    all_df["Embarked"].fillna("missing", inplace=True)

    return all_df

In [10]:
def get_preprocessed_dataset_6(all_df):
    # 카테고리 변수를 LabelEncoder를 사용하여 수치값으로 변경하기
    # 카테고리 변수의 값이 "object"인 모든 열 -> LabelEncoder로 변환할 카테고리 변수의 목록을 생성
    category_features = all_df.columns[all_df.dtypes == "object"]
    from sklearn.preprocessing import LabelEncoder
    # LabelEncoder를 사용해 카테고리 변수를 수치로 변환하고 원래 값을 대체
    for category_feature in category_features:
        le = LabelEncoder()
        if all_df[category_feature].dtypes == "object":
          le = le.fit(all_df[category_feature])
          all_df[category_feature] = le.transform(all_df[category_feature])

    return all_df

### 정리 (3)
데이터 전처리 과정에서 수행되는 각 메소드별 기능은 아래와 같다.
- get_preprocessed_dataset_1 : Fare(운임 요금)의 결측치를 Pclass(티켓 클래스)별 평균 운임 요금으로 대체한다.
- get_preprocessed_dataset_2 : 기존 train.csv 파일의 Name 컬럼은 "Braund, Mr. Owen Harris"와 같다. 이것을 family_name(성), honorific(경칭), name(이름)으로 세분화한다.
- get_preprocessed_dataset_3 : Age(나이)의 결측치를 2번에서 분리한 컬럼 중 honorific(경칭)별 Age(나이)의 중앙값을 계산하고, 결측치를 이것으로 대체한다.
- get_preprocessed_dataset_4 : 부모/자식, 형제/자매 수를 합쳐 가족 수라는 새로운 컬럼을 추가한다. 또한, 이것을 바탕으로 혼자 탑승한 사람을 구분하는 "alone" 컬럼을 추가한다.
                                또한, 학습에 불필요한 (탑승자 ID, Name, 성, 이름, Ticket, 사물함 번호)를 제거한다.
- get_preprocessed_dataset_5 : honorific(경칭) 컬럼의 수를 (Mr, Miss, Mrs, Master)를 제외하고 모두 "other"로 변경한다. 이를 통해 변경되는 데이터 중 "Dr."가 있다. 또한, 승선항구의 결측치를 "missing"으로 대체한다.
- get_preprocessed_dataset_6 : 데이터 타입이 "object"인 모든 카테고리 변수를 수치화한다.

위 코드의 궁극적인 목표는 모델이 학습하기 좋은 형태로 데이터를 전처리하는 데 있다.

In [11]:
from torch import nn

In [12]:
class MyModel(nn.Module):
    def __init__(self, n_input, n_output):
        # nn.Module을 상속받아 MyModel(신경망 모델) 정의
        super().__init__()
        
        ## nn.Seqeuntail -> 순차적으로 Layer를 쌓는다.
        self.model = nn.Sequential(
        # 첫번째 Layer, n_input개의 입력 특성과 30개의 뉴런을 가진 Fully Connected Layer 생성
        nn.Linear(n_input, 30),
        # Activation Function으로 ReLu() 적용
        nn.ReLU(),
        # 두번째 Layer, 30개의 뉴런을 가진 Layer
        nn.Linear(30, 30),
        nn.ReLU(),
        # 세번째 Layer, 출력을 위한 Layer
        nn.Linear(30, n_output),
        )

    # forward 메소드 정의
    def forward(self, x):
        x = self.model(x)
        return x

In [13]:
def test(test_data_loader):
    print("[TEST]")
    # 테스트 데이터의 batch를 가져온다.
    batch = next(iter(test_data_loader))
    print("{0}".format(batch['input'].shape))
    # 모델의 입력 특성(11개), 출력 특성(2개)
    my_model = MyModel(n_input=11, n_output=2)
    # 모델에 테스트 데이터 배치의 input 값을 넣어준다.
    output_batch = my_model(batch['input'])
    
    prediction_batch = torch.argmax(output_batch, dim=1)
    for idx, prediction in enumerate(prediction_batch, start=892):
        print(idx, prediction.item())

In [14]:
if __name__ == "__main__":
    # 데이터 전처리 수행 후 데이터셋을 반환
    train_dataset, validation_dataset, test_dataset = get_preprocessed_dataset()

    # 훈련, 검증, 테스트 데이터셋의 데이터 갯수 확인
    print("train_dataset: {0}, validation_dataset.shape: {1}, test_dataset: {2}".format(
        len(train_dataset), len(validation_dataset), len(test_dataset)
    ))
    print("#" * 50, 1)
    
    # 훈련 데이터셋의 input features의 값과 target의 값 출력
    for idx, sample in enumerate(train_dataset):
        print("{0} - {1}: {2}".format(idx, sample['input'], sample['target']))

    print("#" * 50, 2)

    # DataLoader 생성 -> 배치 사이즈 16, 무작위로
    # 테스트 데이터는 전체 데이터를 하나의 배치로 생성
    train_data_loader = DataLoader(dataset=train_dataset, batch_size=16, shuffle=True)
    validation_data_loader = DataLoader(dataset=validation_dataset, batch_size=16, shuffle=True)
    test_data_loader = DataLoader(dataset=test_dataset, batch_size=len(test_dataset))

    # 훈련 데이터로더의 shape 출력
    # 총 44개의 미니 배치와 남은 9개 배치
    print("[TRAIN]")
    for idx, batch in enumerate(train_data_loader):
        print("{0} - {1}: {2}".format(idx, batch['input'].shape, batch['target'].shape))
    
    # 검증 데이터로더의 shape 출력
    # 총 10개의 미니 배치와 남은 2개 배치
    print("[VALIDATION]")
    for idx, batch in enumerate(validation_data_loader):
        print("{0} - {1}: {2}".format(idx, batch['input'].shape, batch['target'].shape))

    print("#" * 50, 3)

    # 테스트 함수 호출
    test(test_data_loader)

train_dataset: 713, validation_dataset.shape: 178, test_dataset: 418
################################################## 1
0 - tensor([ 3.0000,  1.0000, 29.0000,  0.0000,  0.0000,  7.7500,  1.0000, 13.3029,
         2.0000,  0.0000,  1.0000]): 0
1 - tensor([ 3.0000,  1.0000, 29.0000,  0.0000,  0.0000,  8.0500,  2.0000, 13.3029,
         2.0000,  0.0000,  1.0000]): 0
2 - tensor([ 3.0000,  1.0000, 19.0000,  0.0000,  0.0000, 10.1708,  2.0000, 13.3029,
         2.0000,  0.0000,  1.0000]): 0
3 - tensor([ 1.0000,  1.0000, 37.0000,  0.0000,  1.0000, 29.7000,  0.0000, 87.5090,
         2.0000,  1.0000,  0.0000]): 0
4 - tensor([ 3.0000,  1.0000, 18.0000,  1.0000,  1.0000, 20.2125,  2.0000, 13.3029,
         2.0000,  2.0000,  0.0000]): 0
5 - tensor([  1.0000,   0.0000,  40.0000,   0.0000,   0.0000, 153.4625,   2.0000,
         87.5090,   1.0000,   0.0000,   1.0000]): 1
6 - tensor([ 1.0000,  0.0000, 19.0000,  0.0000,  0.0000, 30.0000,  2.0000, 87.5090,
         1.0000,  0.0000,  1.0000]): 1
7 - te

# 요구사항 2. Titanic 딥러닝 모델 훈련 코드 및 Activation Function 변경해보기

## 2-1. f_my_model_training_with_argparse_wandb.py 변형
f_my_model_training_with_argparse_wandb.py 코드에서 타이타닉 데이터에 맞게 코드를 변형

In [15]:
import torch
from torch import nn, optim
from torch.utils.data import random_split, DataLoader
from datetime import datetime
import wandb
import argparse
import pandas as pd

# BASE_PATH 개발 환경에 맞게 설정
from pathlib import Path
BASE_PATH = str(Path("hw2.ipynb").resolve().parent.parent.parent) 
# BASE_PATH: /Users/sehyeon/Desktop/SEHYEON/2023-2학기/딥러닝및실습/workspace/DL
import sys
sys.path.append(BASE_PATH)

# Titanic_Dataset을 가져온다.
from _00_homework.hw2.a_titianic_dataset import get_preprocessed_dataset

# Titanic Dataset을 전처리하고, DataLoader를 생성해 반환하는 함수
def get_data():
    # 데이터를 전처리하여 데이터셋을 반환하는 get_preprocessed_dataset() 메소드 호출
    train_dataset, validation_dataset, test_dataset = get_preprocessed_dataset()
    print(len(train_dataset), len(validation_dataset), len(test_dataset))
    
    # 데이터로더 설정
    # 훈련 데이터는 shuffle -> 무작위로 batch_size = 512만큼 가지고 온다.
    # 검증 데이터와 테스트 데이터는 데이터 전체를 하나의 배치로 가지고 온다.
    train_data_loader = DataLoader(dataset=train_dataset, batch_size=wandb.config.batch_size, shuffle=True)
    validation_data_loader = DataLoader(dataset=validation_dataset, batch_size=len(validation_dataset))
    test_data_loader = DataLoader(dataset=test_dataset, batch_size=len(test_dataset))

    return train_data_loader, validation_data_loader, test_data_loader

# 모델 정의
class MyModel(nn.Module):
    def __init__(self, n_input, n_output):
        super().__init__()

        # Input Layer, Hidden Layer, Output Layer 구조로 모델을 생성
        # 현재 11/30/2 구조로 모델을 생성한다.
        self.model = nn.Sequential(
            nn.Linear(n_input, wandb.config.n_hidden_unit_list[0]),
            nn.ReLU(),
            nn.Linear(wandb.config.n_hidden_unit_list[0], wandb.config.n_hidden_unit_list[1]),
            nn.ReLU(),
            nn.Linear(wandb.config.n_hidden_unit_list[1], n_output),
            # 생존/사망 분류 문제이므로 Softmax를 이용해 모델의 출력값을 0과 1 사이의 확률 (output 2개 값의 합이 1)로 표현한다.
            nn.Softmax(dim=1)
        )

    def forward(self, x):
        x = self.model(x)
        return x

# 모델과 optimizer를 반환
def get_model_and_optimizer():
    # 모델 생성, input = 11 output = 2
    my_model = MyModel(n_input=11, n_output=2)
    # optimizer는 확률적 경사하강법인 SGD를 사용한다.
    optimizer = optim.SGD(my_model.parameters(), lr=wandb.config.learning_rate)

    return my_model, optimizer

# 모델 학습을 위한 training_loop
def training_loop(model, optimizer, train_data_loader, validation_data_loader):
    n_epochs = wandb.config.epochs
    # 손실함수로 분류 문제에서 사용하는 CrossEntropyLoss 사용
    loss_fn = nn.CrossEntropyLoss()  # Use a built-in loss function
    next_print_epoch = 100

    for epoch in range(1, n_epochs + 1):
        loss_train = 0.0
        num_trains = 0
        # 훈련 데이터 학습
        for train_batch in train_data_loader:
            # 모델의 output 결과 반환
            output_train = model(train_batch['input'])
            # 모델의 output과 실제 target으로 손실함수 호출
            loss = loss_fn(output_train, train_batch['target'])
            loss_train += loss.item()
            num_trains += 1

            # Gradient를 초기화 -> 다음 backward에 영향을 미치지 않게 한다.
            optimizer.zero_grad()
            loss.backward()
            # 가중치 업데이트
            optimizer.step()

        loss_validation = 0.0
        num_validations = 0
        with torch.no_grad():
            # 검증 데이터
            for validation_batch in validation_data_loader:
                # 검증 데이터에 대한 모델의 output 반환
                output_validation = model(validation_batch['input'])
                # 모델의 output과 실제 검증 데이터의 target으로 손실함수 호출
                loss = loss_fn(output_validation, validation_batch['target'])
                loss_validation += loss.item()
                num_validations += 1

        wandb.log({
            "Epoch": epoch,
            "Training loss": loss_train / num_trains,
            "Validation loss": loss_validation / num_validations
        })

        if epoch >= next_print_epoch:
            print(
                f"Epoch {epoch}, "
                f"Training loss {loss_train / num_trains:.4f}, "
                f"Validation loss {loss_validation / num_validations:.4f}"
            )
            next_print_epoch += 100
            
# 학습된 모델을 테스트
def test(test_data_loader, model):
    print("[TEST]")
    # 모델을 평가 모드로 전환한다.
    model.eval()
    # 테스트 데이터의 배치(전체 데이터)를 가지고 온다.
    batch = next(iter(test_data_loader))
    # 모델에 테스트 데이터를 넣어준다.
    output_batch = model(batch['input'])
    # 0과 1로 분류된 확률값 중 더 큰 값의 인덱스를 반환한다.
    # 이것이 모델이 분류한 0(사망), 1(생존) 값이되면 최종 출력값이 된다.
    prediction_batch = torch.argmax(output_batch, dim=1)
    # prediction_batch를 출력을 위해 numpy 배열로 변환
    labels = prediction_batch.numpy()

    passenger_ids = list(range(892, 892 + len(prediction_batch)))
    # submission 데이터프레임 생성
    submission_df = pd.DataFrame({
        'PassengerId': passenger_ids,
        'Survived': labels
    })
    # submission.csv 파일로 저장
    submission_df.to_csv('submission.csv', index=False)


def main(args):
    current_time_str = datetime.now().astimezone().strftime('%Y-%m-%d_%H-%M-%S')

    # ephocs, batch_size, lerning_rate, hidden layer 뉴런의 개수에 대한 설정값
    config = {
        'epochs': args.epochs,
        'batch_size': args.batch_size,
        'learning_rate': 1e-3,
        'n_hidden_unit_list': [30, 30],
    }
    
    # wandb에 저장할 메타데이터 정보
    wandb.init(
        mode="online" if args.wandb else "disabled",
        project="my_model_training",
        notes="My first wandb experiment",
        tags=["my_model", "titanic"],
        name=current_time_str,
        config=config
    )
    print(args)
    print(wandb.config)

    # 데이터 로더를 얻어오는 함수 호출
    train_data_loader, validation_data_loader, test_data_loader = get_data()

    linear_model, optimizer = get_model_and_optimizer()

    wandb.watch(linear_model)

    print("#" * 50, 1)

    # 훈련 시작
    training_loop(
        model=linear_model,
        optimizer=optimizer,
        train_data_loader=train_data_loader,
        validation_data_loader=validation_data_loader
    )
    # 훈련 종료 후 테스트 진행
    test(test_data_loader, linear_model)
    wandb.finish()
    
if __name__ == "__main__":
    parser = argparse.ArgumentParser()

    parser.add_argument(
        "--wandb", action=argparse.BooleanOptionalAction, default=False, help="True or False"
    )

    parser.add_argument(
        "-b", "--batch_size", type=int, default=16, help="Batch size (int, default: 16)"
    )

    parser.add_argument(
        "-e", "--epochs", type=int, default=1_000, help="Number of training epochs (int, default:1_000)"
    )

    args = parser.parse_args("")

    main(args)

Namespace(wandb=False, batch_size=16, epochs=1000)
{'epochs': 1000, 'batch_size': 16, 'learning_rate': 0.001, 'n_hidden_unit_list': [20, 20]}
Data Size: 891, Input Shape: torch.Size([891, 11]), Target Shape: torch.Size([891])
713 178 418
################################################## 1
Epoch 100, Training loss 0.5818, Validation loss 0.5807
Epoch 200, Training loss 0.5779, Validation loss 0.5770
Epoch 300, Training loss 0.5730, Validation loss 0.5777
Epoch 400, Training loss 0.5691, Validation loss 0.5804
Epoch 500, Training loss 0.5632, Validation loss 0.5721
Epoch 600, Training loss 0.5594, Validation loss 0.5694
Epoch 700, Training loss 0.5520, Validation loss 0.5692
Epoch 800, Training loss 0.5453, Validation loss 0.5815
Epoch 900, Training loss 0.5320, Validation loss 0.5505
Epoch 1000, Training loss 0.5210, Validation loss 0.5464
[TEST]


## 2-2. Activation Function 비교
먼저, 훈련 데이터를 가지고 올 때 `shuffle = True` 이므로 랜덤하게 데이터를 가지고와 batch를 구성한다.
따라서, Training Loss가 어느정도 달라질 수 있다.
여기서는 ELU, ReLU, LeakyReLU, PReLU 총 4개의 활성함수의 성능을 비교한다.
- epochs = 2,000 기타 조건은 동일한 상황에서 활성함수만 변경하여 테스트한다.
- 각 활성함수 당 5번의 테스트를 진행한다.
- Training Loss, Validation Loss의 평균을 이용해 성능 분석을 한다.

### 분석결과
---

### 1) ELU

📌 Wandb Link -> [Click -> ELU](https://wandb.ai/noeyhesx/comparison_activation_function_ELU?workspace=user-noeyhesx)

![](https://github.com/BBOXEEEE/Deep_Learning/blob/main/_00_homework/hw2/img/wandb_ELU.png?raw=true)


| **Epochs** | **Training Loss** | **Validation Loss** |
|:----------:|:-----------------:|:-------------------:|
|    2000    |      0.4854       |       0.5091        |
|    2000    |      0.4864       |       0.5305        |
|    2000    |      0.4903       |        0.54         |
|    2000    |      0.4885       |       0.5238        |
|    2000    |      0.4834       |       0.4927        |
|  **AVG**   |    **0.4868**     |     **0.5192**      |

### 2) ReLU

📌 Wandb Link -> [Click -> ReLU](https://wandb.ai/noeyhesx/comparison_activation_function_ReLU?workspace=user-noeyhesx)

![](https://github.com/BBOXEEEE/Deep_Learning/blob/main/_00_homework/hw2/img/wandb_ReLU.png?raw=true)


| **Epochs** | **Training Loss** | **Validation Loss**  |
|:----------:|:-----------------:|:--------------------:|
|    2000    |      0.4847       |        0.5322        |
|    2000    |      0.4937       |        0.5377        |
|    2000    |      0.4882       |        0.5017        |
|    2000    |      0.4973       |        0.5238        |
|    2000    |      0.4952       |        0.477        |
|  **AVG**   |    **0.4918**     |      **0.5109**      |

### 3) LeakyReLU

📌 Wandb Link -> [Click -> LeakyReLU](https://wandb.ai/noeyhesx/comparison_activation_function_LeakyReLU/workspace?workspace=user-noeyhesx)

![](https://github.com/BBOXEEEE/Deep_Learning/blob/main/_00_homework/hw2/img/wandb_LeakyReLU.png?raw=true)


| **Epochs** | **Training Loss** | **Validation Loss**  |
|:----------:|:-----------------:|:--------------------:|
|    2000    |      0.5059       |        0.5071        |
|    2000    |      0.4886       |        0.5304        |
|    2000    |      0.5524       |        0.5668        |
|    2000    |      0.496       |        0.5202        |
|    2000    |      0.4989       |        0.4926        |
|  **AVG**   |    **0.5083**     |      **0.5234**      |

### 4) PReLU

📌Wandb Link -> [Click -> PReLU](https://wandb.ai/noeyhesx/comparison_activation_function_PReLU/workspace?workspace=user-noeyhesx)

![](https://github.com/BBOXEEEE/Deep_Learning/blob/main/_00_homework/hw2/img/wandb_PReLU.png?raw=true)


| **Epochs** | **Training Loss** | **Validation Loss**  |
|:----------:|:-----------------:|:--------------------:|
|    2000    |      0.4956       |        0.5515        |
|    2000    |      0.5102       |        0.5211        |
|    2000    |      0.5045       |        0.526        |
|    2000    |      0.4966       |        0.5137        |
|    2000    |      0.497       |        0.4951        |
|  **AVG**   |    **0.5007**     |      **0.5214**      |

### 선택
- ReLU 기반으로 만들어진 손실함수이기 때문에 큰 차이가 나지는 않았다.
- 하지만, 연산량과 훈련-검증 loss의 차이 등을 고려해서 ReLU를 타이타닉 모델의 활성함수로 사용한다.

# 요구사항 3. 테스트 및 submission.csv 생성

## 3-1. 선택한 Activation Function으로 모델 구성
위에서 분석한 내용을 Activation Function은 **ReLU**를 이용한다.

In [19]:
import torch
from torch import nn, optim
from torch.utils.data import random_split, DataLoader
from datetime import datetime
import wandb
import argparse
import pandas as pd

# BASE_PATH 개발 환경에 맞게 설정
from pathlib import Path
BASE_PATH = str(Path("hw2.ipynb").resolve().parent.parent.parent) 
# BASE_PATH: /Users/sehyeon/Desktop/SEHYEON/2023-2학기/딥러닝및실습/workspace/DL
import sys
sys.path.append(BASE_PATH)

# Titanic_Dataset을 가져온다.
from _00_homework.hw2.a_titianic_dataset import get_preprocessed_dataset

# Titanic Dataset을 전처리하고, DataLoader를 생성해 반환하는 함수
def get_data():
    # 데이터를 전처리하여 데이터셋을 반환하는 get_preprocessed_dataset() 메소드 호출
    train_dataset, validation_dataset, test_dataset = get_preprocessed_dataset()
    print(len(train_dataset), len(validation_dataset), len(test_dataset))
    
    # 데이터로더 설정
    # 훈련 데이터는 shuffle -> 무작위로 batch_size = 512만큼 가지고 온다.
    # 검증 데이터와 테스트 데이터는 데이터 전체를 하나의 배치로 가지고 온다.
    train_data_loader = DataLoader(dataset=train_dataset, batch_size=wandb.config.batch_size, shuffle=True)
    validation_data_loader = DataLoader(dataset=validation_dataset, batch_size=len(validation_dataset))
    test_data_loader = DataLoader(dataset=test_dataset, batch_size=len(test_dataset))

    return train_data_loader, validation_data_loader, test_data_loader

# 모델 정의
class MyModel(nn.Module):
    def __init__(self, n_input, n_output):
        super().__init__()

        # Input Layer, Hidden Layer, Output Layer 구조로 모델을 생성
        # 현재 11/30/2 구조로 모델을 생성한다.
        self.model = nn.Sequential(
            nn.Linear(n_input, wandb.config.n_hidden_unit_list[0]),
            nn.ReLU(),
            nn.Linear(wandb.config.n_hidden_unit_list[0], wandb.config.n_hidden_unit_list[1]),
            nn.ReLU(),
            nn.Linear(wandb.config.n_hidden_unit_list[1], n_output),
            # 생존/사망 분류 문제이므로 Softmax를 이용해 모델의 출력값을 0과 1 사이의 확률 (output 2개 값의 합이 1)로 표현한다.
            nn.Softmax(dim=1)
        )

    def forward(self, x):
        x = self.model(x)
        return x

# 모델과 optimizer를 반환
def get_model_and_optimizer():
    # 모델 생성, input = 11 output = 2
    my_model = MyModel(n_input=11, n_output=2)
    # optimizer는 확률적 경사하강법인 SGD를 사용한다.
    optimizer = optim.SGD(my_model.parameters(), lr=wandb.config.learning_rate)

    return my_model, optimizer

# 모델 학습을 위한 training_loop
def training_loop(model, optimizer, train_data_loader, validation_data_loader):
    n_epochs = wandb.config.epochs
    # 손실함수로 분류 문제에서 사용하는 CrossEntropyLoss 사용
    loss_fn = nn.CrossEntropyLoss()  # Use a built-in loss function
    next_print_epoch = 100

    for epoch in range(1, n_epochs + 1):
        loss_train = 0.0
        num_trains = 0
        # 훈련 데이터 학습
        for train_batch in train_data_loader:
            # 모델의 output 결과 반환
            output_train = model(train_batch['input'])
            # 모델의 output과 실제 target으로 손실함수 호출
            loss = loss_fn(output_train, train_batch['target'])
            loss_train += loss.item()
            num_trains += 1

            # Gradient를 초기화 -> 다음 backward에 영향을 미치지 않게 한다.
            optimizer.zero_grad()
            loss.backward()
            # 가중치 업데이트
            optimizer.step()

        loss_validation = 0.0
        num_validations = 0
        with torch.no_grad():
            # 검증 데이터
            for validation_batch in validation_data_loader:
                # 검증 데이터에 대한 모델의 output 반환
                output_validation = model(validation_batch['input'])
                # 모델의 output과 실제 검증 데이터의 target으로 손실함수 호출
                loss = loss_fn(output_validation, validation_batch['target'])
                loss_validation += loss.item()
                num_validations += 1

        wandb.log({
            "Epoch": epoch,
            "Training loss": loss_train / num_trains,
            "Validation loss": loss_validation / num_validations
        })

        if epoch >= next_print_epoch:
            print(
                f"Epoch {epoch}, "
                f"Training loss {loss_train / num_trains:.4f}, "
                f"Validation loss {loss_validation / num_validations:.4f}"
            )
            next_print_epoch += 100
            
# 학습된 모델을 테스트
def test(test_data_loader, model):
    print("[TEST]")
    # 모델을 평가 모드로 전환한다.
    model.eval()
    # 테스트 데이터의 배치(전체 데이터)를 가지고 온다.
    batch = next(iter(test_data_loader))
    # 모델에 테스트 데이터를 넣어준다.
    output_batch = model(batch['input'])
    # 0과 1로 분류된 확률값 중 더 큰 값의 인덱스를 반환한다.
    # 이것이 모델이 분류한 0(사망), 1(생존) 값이되면 최종 출력값이 된다.
    prediction_batch = torch.argmax(output_batch, dim=1)
    # prediction_batch를 출력을 위해 numpy 배열로 변환
    labels = prediction_batch.numpy()

    passenger_ids = list(range(892, 892 + len(prediction_batch)))
    # submission 데이터프레임 생성
    submission_df = pd.DataFrame({
        'PassengerId': passenger_ids,
        'Survived': labels
    })
    # submission.csv 파일로 저장
    submission_df.to_csv('submission.csv', index=False)


def main(args):
    current_time_str = datetime.now().astimezone().strftime('%Y-%m-%d_%H-%M-%S')

    # ephocs, batch_size, lerning_rate, hidden layer 뉴런의 개수에 대한 설정값
    config = {
        'epochs': args.epochs,
        'batch_size': args.batch_size,
        'learning_rate': 1e-3,
        'n_hidden_unit_list': [30, 30],
    }
    
    # wandb에 저장할 메타데이터 정보
    wandb.init(
        mode="online" if args.wandb else "disabled",
        project="my_model_training",
        notes="My first wandb experiment",
        tags=["my_model", "titanic"],
        name=current_time_str,
        config=config
    )
    print(args)
    print(wandb.config)

    # 데이터 로더를 얻어오는 함수 호출
    train_data_loader, validation_data_loader, test_data_loader = get_data()

    linear_model, optimizer = get_model_and_optimizer()

    wandb.watch(linear_model)

    print("#" * 50, 1)

    # 훈련 시작
    training_loop(
        model=linear_model,
        optimizer=optimizer,
        train_data_loader=train_data_loader,
        validation_data_loader=validation_data_loader
    )
    # 훈련 종료 후 테스트 진행
    test(test_data_loader, linear_model)
    wandb.finish()
    
if __name__ == "__main__":
    parser = argparse.ArgumentParser()

    parser.add_argument(
        "--wandb", action=argparse.BooleanOptionalAction, default=False, help="True or False"
    )

    parser.add_argument(
        "-b", "--batch_size", type=int, default=16, help="Batch size (int, default: 16)"
    )

    parser.add_argument(
        "-e", "--epochs", type=int, default=7_000, help="Number of training epochs (int, default:1_000)"
    )

    args = parser.parse_args("")

    main(args)

Namespace(wandb=False, batch_size=16, epochs=7000)
{'epochs': 7000, 'batch_size': 16, 'learning_rate': 0.001, 'n_hidden_unit_list': [30, 30]}
Data Size: 891, Input Shape: torch.Size([891, 11]), Target Shape: torch.Size([891])
713 178 418
################################################## 1
Epoch 100, Training loss 0.5929, Validation loss 0.5750
Epoch 200, Training loss 0.5848, Validation loss 0.5702
Epoch 300, Training loss 0.5817, Validation loss 0.5664
Epoch 400, Training loss 0.5815, Validation loss 0.5624
Epoch 500, Training loss 0.5756, Validation loss 0.5577
Epoch 600, Training loss 0.5718, Validation loss 0.5551
Epoch 700, Training loss 0.5652, Validation loss 0.5527
Epoch 800, Training loss 0.5567, Validation loss 0.5485
Epoch 900, Training loss 0.5481, Validation loss 0.5415
Epoch 1000, Training loss 0.5397, Validation loss 0.5322
Epoch 1100, Training loss 0.5311, Validation loss 0.5232
Epoch 1200, Training loss 0.5214, Validation loss 0.5153
Epoch 1300, Training loss 0.5107, 

In [18]:
# 테스트 결과 출력
submission_csv = pd.read_csv("./submission.csv")
print(submission_csv)

     PassengerId  Survived
0            892         0
1            893         0
2            894         0
3            895         0
4            896         0
..           ...       ...
413         1305         0
414         1306         1
415         1307         0
416         1308         0
417         1309         1

[418 rows x 2 columns]


## 3-2. 훈련과정 중 어느 Epoch 시점에 테스트를 수행하여 submission.csv를 구성해야 하는지 고찰하기

적절한 Epoch 시점을 찾기 위해 학습을 많이 시켜도 보고 Kaggle에 결과를 업로드해 정확도를 살펴보기도 했었다.
실제로 Epoch 값을 크게 잡고 반복해서 학습을 하다보면 Training Loss는 점점 감소하는 추세를 보였다.
하지만, 어느 순간 Validation Loss는 감소하지 않고 오히려 증가하며, Training Loss와의 차이가 점점 커지는 것을 발견했다.
약, 40,000번 정도의 Epoch를 수행했을 때 Training Loss와 Validation Loss의 차이가 0.3 이상까지 벌어지는 것을 확인했다.

처음에는 단순히 Epoch가 크면 조금 더 모델이 정확하게 예측을 하지 않을까? 라고 생각했었다.
Kaggle의 정확도도 대체로 Epoch를 크게 잡고 학습한 모델에서 얻어낸 결과가 높았기 때문이다.
문득, 수업시간에 교수님께서 하신 말씀이 떠올랐다. "적당함" "경험적"
모델의 레이어의 개수, 뉴런의 개수, Learning Rate 등등
이와 비슷한 애매모호한, 직접 해보지 않으면 느낄 수 없는 것들의 정답은 적당함이었다.

Epoch를 찾는 과정에서 적당한 값이 무엇일까 고민을 많이 해봤지만, 명확하게 이 시점에서 해야된다는 정의하지 못할 것 같다.
다만 몇가지 공통적으로 발생하는 것을 찾았다면 다음과 같다.
- 너무 작은 epoch에서는 정확한 결과를 도출해내지 못한다.
- ReLU는 생각보다 빠르게 Loss 값이 떨어지고 천천히 감소한다.
- 안정화되는 영역을 찾는 것이 중요하다.
- 너무 많은 학습을 시키면 훈련 데이터에만 익숙해지는 Overfitting 문제가 발생한다. 반대로 너무 작은 epoch는 Underfitting 문제가 생긴다.

이것을 바탕으로 epoch 값은 7,000으로 설정하고 정해진 `training_loop`가 끝난 후 테스트를 수행하도록 코드를 작성했다.

# 요구사항 4. submission.csv 제출 및 등수확인
![](https://github.com/BBOXEEEE/Deep_Learning/blob/main/_00_homework/hw2/img/kaggle_score.png?raw=true)

# 요구사항 5. Wandb 페이지 생성 및 URL 제출
### Wandb Project Page
아래 링크는 Public으로 열어놓은 4개의 활성 함수 테스트를 위해 생성한 프로젝트가 있다.
또한, comparision_activation_function_ReLU 프로젝트에는 ReLU를 사용한 모델의 epoch 7,000회 테스트 결과가 추가로 있다.
[https://wandb.ai/noeyhesx/projects](https://wandb.ai/noeyhesx/projects)

# 숙제 후기
## ☀️ 고찰
### 깨달음의 과정
과제를 시작하면서 타이타닉은 어떤 유형의 문제일까 고민했었다. california_housing_dataset을 이용한 Regression 문제일까, Classification 문제일까 고민이 많았었다.
분명, 제공된 titanic_dataset.py 파일에서 모델을 (11, 2)와 같이 output layer를 2개 생성하는 것을 보면 분류 문제 같았다. 이렇게 갈팡질팡하면서 코드를 작성하고 테스트를 실행하고 캐글에 제출한 결과는 당연히 좋지 못했다.
수업 시간에 장난으로 gender_submission.csv 파일이 여자는 살고, 남자는 죽는다고 표현한 그 데이터로 제출한 것보다 훨씬 낮았다. 그래서 손실함수를 건드려보았다.
MSELoss() 에서 CrossEntropyLoss()로 변경을 했었다. MSELoss()를 사용했을 땐, train_batch의 타겟을 손실함수에 넣는 과정에서 차원을 확장하고 float화 했던 과정을 안해도 되었었다.
약간 합리적인 의심으로 접근했던 것이었다. 정확한 의미도 모른 채..

그래서 데이터셋을 자세히 들여다보기로 했다. 인터넷에 많은 사람들이 타이타닉 데이터를 분석한 것들도 참고하며 분석한 결과, gender_submission.csv가 높았던 이유도 알게 되었다.
당시 여성의 생존률이 매우 높았고, 결과에 영향을 주는 column이 성별이 압도적으로 높은 수치였다는 것을 알게 되었다.
딥러닝 과정에서 무작정 모델을 구성하고 데이터를 밀어넣는 것보다 데이터를 전처리하고 분석하는 과정도 중요하다는 것을 새삼 깨닫게 되었다.
그와 동시에 titanic_dataset에서 학습에 불필요한 컬럼을 제거한 이유도 더욱 와닿았다.

이제는 손실함수를 왜 그것을 써야하는지 고민했다. 이 과정에서 강의자료를 계속 읽어보았다. 숫자 이미지를 분류하는 문제에서 분류 문제는 CrossEntropyLoss를 사용하고, Regression 문제는 MSELoss를 사용한다고 했다.
강의자료를 먼저 쭉 공부하고 시작했더라면 이런 삽질은 안했을 것이라 생각이 들었다. 분명, 수업시간에 들었던 기억이 나면서 복습을 더 해야겠다 생각했다.
그러면서 이 문제는 주어진 탑승객의 정보를 바탕으로 생존/사망을 분류하는 문제라는 생각을 굳히고 나서는 어느정도 과제 진행에 속도를 낼 수 있었다.

어느정도 모델이 구성되고 학습이 진행되고 결과를 눈으로 확인할 수 있어지자 캐글 스코어를 높혀보고 싶은 욕심이 생겼다.
이 과정에서 다양한 시도들을 해보았다. 레이어의 개수를 늘리면? 뉴런의 개수를 늘리면? epoch를 높게 설정하면?
다 늘린다고 장땡은 아니였고, 이런 시도들 덕분에 Overfitting, Underfitting 등 모델 학습 시 주의해야하는 점에 대해서 더욱 알게 되었다.

이러한 시도들을 통해 FCN에 대한 이해가 부족했음을 깨닫고, 이론을 다시 한 번 들여다보고 적용하면서 얻어가는 점이 분명히 있었다고 생각한다.
남는 것이 꽤 많은 삽질의 과정이었다.

## ☁️ 후기
사실 중간고사 기간이라 과제를 빨리 끝내고 다른 과목 공부에 집중하려 수업 시간에 배운 내용을 조금 더 자세히 들여다보지 않고 성급하게 과제를 수행한 면도 있는 것 같다.
그렇게 과제를 빨리 끝내고자 한 시작은 결국에 삽질이 되어 다시 공부하면서 차근차근 과제를 수행해 나가게 되었다.
나름대로 고민하고 잘 해결이 안되서 교수님께 메일로 질문을 드렸었는데 다시보니 이해가 부족한 상황에서 한 질문같아 스스로 조금 부끄럽기도 하다.
교수님께서 수업 시간에 굉장히 좋은 코드를 가지고 공부한다고 말씀하신 적이 있다.
그렇게 말씀하신 이유를 깨달았다.
과제를 하면서 과제를 올바르게 끝내려다 보면 자연스레 공부를 하게 되고, 깨닫는 것이 생긴다.
수업 시간에 배운 이론들을 다시 공부하고 적용하다보면 이게 되네? 이렇게 되는거였구나 할 때마다 조금 힘들어도 원동력이 생기는 것 같다.